![figure](../lab1/lab1_figures/politecnico_h-01.png)
# **Eletrónica Configurável / Configurable Electronics**
#### Mestrado em Engenharia Eletrotécnica / Master in Electrical and Electronic Engineering

## **LabWork4 - Xilinx IP Cores for DSP**

_____________ 

## Introduction ##
In this lab you will explore serial and parallel filter implementation using the FIR compiler IP core from Xilinx, with an AXI interface. For data generation you will resort to a ROM initializated with an audio sample. Results will be observed using an Integrated Logic Analyser (ILA).

### Objectives ###
After completing this lab, you will be able to:

* Initialize and configure ROM IP core
* Initialize and configure FIR Compiler IP core and its AXI interface
* Understand the concept of serial/parallel hardware implementation 
* Use advanced ILA features to test and debug the design
* Experiment audio generation with Digilent PmodDA2 and PmodAmp1


In the instructions below **{sources}** refers to `C:\Xilinx\MEE_EC\sources` and **{labs}** refers to `(C:\Xilinx\MEE_EC\labs)`

### Design description ###

The block diagram shown in the following figure shows the high level structure of the circuit we will design. It includes two RTL modules described in `.vhd` and some IP cores from xilinx library:
* DA2 reference component module `DA2RefComp.vhd` to interface with PmodDA2 board; 
* Edge detection module `edge_detect.vhd` to generate the *start transmission* signal;
* ROM IP module to generate an audio sample; 
* FIR Compiler IP module to filter the signal;
* Clock Synthesis module to generate the necessary clock signals;
* Integrated Logic Analyser;

Also provided, there is a XDC file with I/O constraints for implementation:
* Constraints file is `lab4.xdc`.

![Figure](../lab4/lab4_figures/fig1.png)


The audio sample was be generated in Matlab, as a weighted sum of four sine signals, using the code shown below: 

```
fs = 44100;                       % Sampling frequency (samples per second)
dt = 1/fs;                        % seconds per sample
StopTime = 2;                     % seconds
t = (0:dt:StopTime-dt)';          % seconds
F = [220, 440, 5000, 10000];      % Sine wave frequencies (hertz)
sine1 = sin(2*pi*F(1)*t);     
sine2 = sin(2*pi*F(2)*t);
sine3 = sin(2*pi*F(3)*t);   
sine4 = sin(2*pi*F(4)*t);
% weighted sum of sine waves
sine=(0.5*sine1+0.3*sine2+0.10*sine3+0.10*sine4); 
figure, plot(sine(1:200));
% limit depth to 2^16 and absolute value to 2^11
sineout=floor(sine(1:2^16)*2^11);    
% save in file (must be converted to .COE with Notepad++)
csvwrite('sinesum.dat',sineout);     
```

_____

## Step 1 - Create a Vivado Design ##

### Step 1.1 ###

In this section you will create a new and empty project for PYNQ-Z2 Board in Vivado. We will use Vivado 2020.1 - note that some options may vary depending on the version you are using.

* Start downloading the source files provided (Moodle) and save them in `{sources}/lab4`.


* Open Vivado and create a new project to start the wizard. Click **Next**.


* Click the browse button of the project location field of the new project form, browse to `c:/Xilinx/MEE_EC/labs` and click **Select**. Enter `lab4` in the project name field. Make sure that the *Create Project Subdirectory* box is checked. Click **Next**.


* We will create an empty RTL project and specify sources later. Select **RTL Project** option in the *Project Type* form, and click **Next**. Make sure that the *Do not specify sources* at this time box is checked.


* In the *Default Part* form, select the *Boards* option and choose the **PYNQ-Z2** board. Click **Next** and then **Finish** to create the Vivado project.


* In the *Sources* pane select **Add Sources** by clicking the **plus** sign. Then select *Add or create design sources* and click **Next**.


* Select *Add Files* and browse to the `{sources}\lab4` directory and select simultaneously the files `DA2refComp.vhd` and `level_shifter.vhd`. Click **OK** and then **Finish**.


* Repeat the last two steps but now select *Add or create constraints* and click **Next**. Select *Add Files* and browse to the `{sources}\lab4` directory, select `lab4.xdc`, click **OK** and then **Finish**. 


### Step 1.2 ### 

In this section you will create the clock synthesis module.


* In the *Flow Navigator*, expand *IP INTEGRATOR* and select **Create Block Design**. Name it **top_lab4** anc click **OK**. 


* We will start by designing the clock synthesis module that will provide different clock frequencies and reset signals to the system. Add the following Xilinx IP cores to the design canvas:
    * 1 Clocking Wizard
    * 1 Utility Vector Logic
    * 1 Processor System Reset
    * 1 Binary Counter
    * 1 Slice
    * 1 Utility Buffer
    
      
* Double click the **Clocking Wizard** to configure it. The input clock should be set to **125MHz**, from which one other clock should be generated at a frequency of **45.1584MHZ**. This frequency will be divided by 1024, outside the clocking wizard module, to obtain a 44.1KHz clock for audio sample generation (the module instanciated by the wizard cannot generate frequencies below a few MHz).  


* Double-click the **Utility Vector Logic** and select **not** with **C_SIZE=1**. This will be necessary to invert the input reset signal because the clocking wizard has this input *active high*, while the Processing System Reset module requires an *active low* reset input.


* Double-click the **Processing System Reset** and set:
    * Ext Reset Active Width = 1
    * Aux Reset Active Width = 1


* Double-click the **Binary Counter** and select an **Output Width = 10** and one **Clock Enable (CE)** input. The MSB of this counter will oscillate at a frequency of 45,1584/2014 = 44100Hz.


* Double-click the **Slice** and select **Din Width = 10** from 9 downto 9 (Dout Width=1).


* Double-click the **Utility Buffer** and select **BUFG**.


* Make the connections, create the external ports and change the names according to the figure.

![Figure](../lab4/lab4_figures/fig2.png)


* Now select all the modules simultaneously (SHIFT + LEFT-CLICK), **righ-click** on them and select **Create Hierarchy** from the drop-down menu. Call it **clock_synth**. This creates an Hierarchical module with these internals, which is responsible for generating:
    * A **45MHz clock** signal necessary for the External **PmodDA2** board and **ILA**.
    * A **44.1KHz clock** to be used as the sample frequency to read the memory instantiated with an audio signal.
    * A **reset** signal, synchronous with the generated clocks, to avoid timing problems.
    * A **Clock enable (CE)** signal indicating that the MMCM (instantiated by the clocking wizard) is locked (is working correctly).
    * A port for the clock divider counter output (**clkdiv**) in order to be able to inspect it latter with the ILA. 
    
    
* Delete all the output ports.

### Step 1.3 ### 

In this section you will create the audio signal generation circuit, to be read at the rate of 44100 samples per second, and connect it to the external PmodDA2 board.
     
    
* In the top level design canvas, add the following Xilinx IP cores:
    * 1 Block Memory Generator
    * 1 Binary Counter
    * 1 Slice module
    * 1 RAM-based Shift Register
    * 1 ILA (Integrated Logic Analyzer)
  

* Double click the **Block Memory Generator** and set the following:
    * Basic tab -> Mode: Stand Alone
    * Basic tab -> Memory Type: Single Port ROM
    * Port A Options tab -> Port A width: 16
    * Port A Options tab -> Port A Depth: 65536
    * Other Options tab -> Check the box: Load Init File box and browse to `{sources}\lab4\sine.coe`
    * Other Options tab -> Check the box: Fill Remaining Memory Locations box with 0

<div class="alert alert-block alert-info">
<b>Info:</b> the COE file is quite big (65536 values) so it may take some time to configure the IP. If it is not responding, just wait a little bit longer.
</div>


* Select the **Binary Counter** module and change its name to **address_gen_0**, in the *Block Properties* window. Now double-click it and configure it to be a 16 bits free running counter (count up to 65535, which is the memory depth). Add a **Clock Enable** input signal.


* Double-click the **Slice** module, name it **data_slice_0** and configure it to have a **DIN Width=16** and the output to be **12 bits** from bit 11 downto bit 0.


* Double click the **RAM-based Shift Register**, name it **start_gen_0** and set the following:
    * Configuration tab -> Shift Register Type: Fixed Lenght
    * Configuration tab -> Dimensions: Width: 1 | Depth: 1024
    * Initialization tab -> Used COE file: tick
    * Initialization tab -> Initialization File: browse to `{sources}\lab4\start.coe`
    

* Double-click the **ILA** and configure it as **Native** with **5 probes**, sample depth equal to 1024, **4 comparators** per probe, **Capture Control** and **Advanced Trigger**. The probe ports should have the sufficient number of bits to be connected to, so go to the Probe_Ports tab and configure the following:
    * The ROM address generator output (16-bit counter output)
    * The ROM data output (16-bit data output)
    * The Data vector sent to external PmodDA2 (12 bits)
    * The edge detector tick signal, i.e., the DA2RefComp start signal (1 bit)
    * The clockdivider counter output (10 bits)
    

* Drag the RTL sources **DA2refComp** and **level_shifter** to the design canvas. The level shifter will introduce a half-scale offset to the signal to make it unipolar, as the pmodDA2 board is powered by a single 3.3V supply.


* **Double-click** on the **RST** input of the `DA2RefComp_0` module and set the polarity to **ACTIVE HIGH**.


* Complete the connections and port renaming until you have the following design:

![Figure](../lab4/lab4_figures/fig3.png)


* **Validate** the design and **regenerate** it in order to see all the modules organized.


* Finally, in the *Sources* window **righ-click** the **top_lab4** block design, and select **Create HDL Wrapper**.

________________

## Step 2 - Implement and Test the Design ##

### Step 2.1 ### 

On this step you will implement the design and observe the generated waveform using ILA and oscilloscope.
     
    
* Make sure all the input and output ports have correspondent names in **lab4.xdc** file. Make changes if necessary.


* In the *Flow Navigator* select **Generate Bitstream**. Vivado will run all the required previous steps (Synthesis and Implementation).


* **Turn on** the PYNQ-Z2 board and connect the USB port to your PC. Make sure you have the mode jumper set to **JTAG** and select **Open Hardware Manager** when implementation finishes. 


* On the **green ribbon** select **Open target > Auto Connect** and once connected select **Program device**. Click **Program** and the **ILA** waveform window should open.


* Select the signals on the waveform window and in the **Debug Probe Properties** window find **Display name** and select **Short name**.


* Right click on the **data out** signals in the waveform window and configure them as **Analog** style and a **Signed Decimal** representation. Maximize the ILA window in order to view a larger waveform window if necessary.


* In the **Settings** tab below the waveform window select:
    * Trigger mode: BASIC_ONLY
    * Capture mode: BASIC
    * Trigger position in window: 100


<div class="alert alert-block alert-info">
<b>Info:</b> Basic trigger means that you trigger the aquisition the moment you press the **run** button. We will explore advanced options latter in the lab.
</div>

<div class="alert alert-block alert-info">
<b>Info:</b> Basic Capture mode means that we need to define the exact moments we need the signal to be captured. As samples are generated at a rate of 44100Hz, we need to capture samples at this same speed!
</div>

* In the **Capture Setup** window, click the **plus** sign to add the **clkdiv** probe. You want to capture samples every time the most significant bit of clkdiv rises. To do that set the radix to **Binary** and the value to **RX_XXXX_XXXX**.


* In the menu bar above the waveform window select **Run trigger for this ILA core**. You should be able to see both waveforms with exactly the same shape. The waveforms are captured at every address change, i.e., at the rate of 44100 sampples per second.

![Figure](../lab4/lab4_figures/fig4.png)


* Observe the waveform using a scope connected to the PmodDA2 outputs. You can also listen if you have headphones and a power amplifier board (PmodAmp1) connected to PmodDA2. 

### Step 2.2 ### 

Capture instants were defined at the rising edge of the **clkdiv MSB**, i.e., at a rate of 44100Hz. However, you may need to capture longer time periods and may not require that much resolution to visualize results, so you can also define multiple capture settings. Lets explore this.


* In the **Capture Setup** window add **the adress_gn_Q** signal, set it to **Binary** and configure the value to even combinations only (**XXXX_XXXX_XXXX_XXX0**). Note that you can now observe a longer signal span, although with a lower resolution.


* **Zoom in** and see that all samples are captured when the **adress_gen_Q** is even. 


<div class="alert alert-block alert-info">
<b>Info:</b> By default, analog waveforms are set to linear interpolate mode, so you cannot see the digital nature of the signals (quantization steps). To see that you have to righ-click **xlslice_0_Dout** and select **Waveform Style > Analog Settings**. Change the Interpolation style to **Hold** and zoom in. Now you can see the actual samples outputed by the ROM at even adresses.
</div>

* If you want to further subsample the analog waveforms you can change the capture value. For example set these values in the address generator output and inspect the captured waveforms:
    * if you set it to XXXX_XXXX_XXXX_XX1X, you will capture two consecutive samples and jump two sample times
    * if you set it to XXXX_XXXX_XXXX_XXRX, you will capture one in every 4 sample times

     
<div class="alert alert-block alert-info">
<b>Info:</b> Values can be **X** (don't care), **0** (zero), **1** (one), **R** (rising), **F** (falling) or **B** (both).
</div>


* Add another condition to capture only positive values of **xlslice_0_Dout**. To do that **add** this signal to the *Capture Setup* window, select **Operator: <**, set the probe to **Binary** and edit the probe value to **0111_1111_1111**. remember that values are in two's complement and thus, negative numbers start with '1'.


* **RUN Trigger** and check results.


* Remove the last two capture conditions and go back to a rising MSB on clkdiv only.


### Step 2.3 ### 

We will now explore trigger settings. In the previous steps, trigger was activated the moment you pressed the button. For this signal that is reasonable because it is periodic and it doesn't really matter which time window is displayed. However, this is not true for most signals in real systems and we need to control the trigger instant. 


* In the **Trigger Setup** window add the **xlslice_0_Dout** signal and set it to the maximum waveform value, i.e., **1723** (radix Signed Decimal). This means the the captured will be triggered when the signal is maximum. Because we set the trigger position in window to be at sample 100, the analog waveform will have the maximum value at this point. **Run Trigger** and confirm this. 


* In the *Settings* window reduce the **window depth to 256** and the **trigger position to 100**. Now the window shows around one period of the waveform. 


* In the **Settings** window (lower left corner of ILA window) set the *Trigger mode* to **ADVANCED_ONLY** signal. In this mode, a *Trigger State Machine* file must be specified. Open the provided file by clicking **Open existing trigger state machine** in the *Trigger Setup*  for this ILA. 


* **Inspect** the state machine code. It can be described as follows:
    * In state 0, reset a counter and wait for the Dout signal to reach the value 1723 (binary 0110 1011 1011)
    * Jump to state 1 when Dout = 1723.
    * In state 1, increment the counter at each sample time (clkdiv MSB rising edge) if the sample is negative (MSB is one)
    * When the counter reaches 6 trigger.
    

<div class="alert alert-block alert-warning">
<b>Note:</b> The condition to increment the counter only at sample times is necessary because the ILA is clocked at ~45MHz, so that is also the finite state machine clock. Because samples are at a slower pace, you have to add this condition to avoid the conter to increment thousands of times during a single negative sample.
</div>
 
 
* Make sure you are looking at **xlslice_0_Dout** without linear interpolation (set it to **hold** in the *Analog Settings* window). **Run Trigger** and **Zoom in**. Use the marker and the keyboard arrows to navigate through samples. Observe that Dout has negative samples at time 89, 90, 91, 95, 98 and 99, after which trigger occurs, i.e., trigger takes place after the 6th negative value of Dout as expected.

![Figure](../lab4/lab4_figures/fig5.png)


<div class="alert alert-block alert-info">
<b>Info:</b> Trigger State machines can be very complex, depending on what you need. You can define multiple counters and use several comparators per probe in the conditional statements. The number of comparisons was set in the **ILA** configuration window. In step 1.3 you configured up to 4 comparators per probe, which is usually more than enough.
</div>


* **Close** the *Hardware Manager* in the blue ribbon and **turn-off** the board.

________________

## Step 3 - Add a filter to your design ##

### Step 3.1 ###

In this step you will introduce a filter to remove the high frequency components of the generated signal. 


* Select **Open Block Design** in the *Flow Navigator* and add a new IP core from Xilinx library - a **FIR Compiler**.


* Double click the **FIR Compiler** to configure it. Make the following changes
    * Filter Options tab > Select source: COE File
    * Filter Options tab > Coefficient File > Browse to file `coefs.coe` in `{sources}\lab4`
    * Channel Specifications tab > Hardware Oversampling Specification > 
        * Select format: Input Sample Period
        * Sample Period: 1
    * Implementation tab > Data Path Options > 
        * Output Rounding Mode: Truncate LSBs
        * Output Width: 16
        

* Browse the tabs to check the available options but leave all other parameters as default. We will be creating a **single rate, single channel low pass filter** with 32 coefficients defined in the COE file (LPF with a 2KHz cuttoff frequency), working with a clock frequency equal to the sample frequency and thus, with a fully parallel architecture. Click **OK**.


* The **FIR Compiler** has a simple **AXI Interface**. You need to provide data and also a **data valid** signal along with the clock. Because the clock will have the same frequency as the sample time, data valid can be set to one. Add a **Constant** IP, set it to **1** and connect it to the data valid input.


* Connect the **aclk** input to the 44.1KHz clock and the **tdata** input to the block memory output (**douta**).


* Add a **slice** and connect the most signifcant bits of the filter's output to the DA2RefComp **DATA2 input** (disconnect it from DATA1 input first)


* Disconnect **ILA probe0** and connect the filter's **tdata** output.


* Disconnect **ILA probe3** and connect it to the filter's **tvalid** output.


* **Validate** the design and **Regenerate Layout**.

![Figure](../lab4/lab4_figures/fig7.png)


* Save the design and **Generate Bitstream**. When this process completes, program de board and observe the signals in ILA and in osciloscope. The filter is supposed to remove the 5MHz and 10MHz signal components.

![Figure](../lab4/lab4_figures/fig8.png)

* **Close** the *Hardware Manager* in the blue ribbon and **turn-off** the board.

### Step 3.2 ###

Explore parallel vs. serial filter implementation


* In the *Project Manager* view open the **Project Summary** (**$\sum$** symbol in the top menu). Check the post implementation **Utilization** graph to see the used resources. Check that information out in the **Table** mode too. Note that this implementation is using **16 DSP** for a 32 tap Filter. Do you know why?


<div class="alert alert-block alert-info">
<b>Info:</b> If you inspect the coefficiens in the COE file, you will notice that they are symmetric!
</div>


* In the *Flow Navigator* select **Open Block Design** and double-click the **FIR Compiler**. In the **Implementation** tab note we have selected **Inferred** Coefficient Structure. As the coeficients are symmetric, the tool took advantage of that to reduce the number of multipliers by half. Select **Non Symmetric** and click **OK**.


* **Validate the Design** and then **Save** it. 


* Select **Run Implementation** and follow all steps to start this process. 


* When it finishes click **Cancel**, as we don't need to open the implemented design. Instead, close the **Block Design** view and check **Project Summary** again in the *Project Manager* view. Note that **32 DSPs** are now used to implement the same 32 coefficient filter because the tool is not taking advantage of the design symmetry.


* **Open Block Design** again and double-click the **FIR Compiler**.  In the **Implementation** tab select **Inferred** Coefficient Structure, allowing the tool to take advantage of coefficient symmetry, if it exists.


* In the **Channel Specification** tab, we will now configure the **Hardware Oversampling Specification**. Make the following changes and then click **OK**:
    * Select format: Frequency Specification
    * Input Sampling Frequency (MHz): 0.0441
    * Clock Frequency (MHz): 45.1584
 
<div class="alert alert-block alert-info">
<b>Info:</b> You could also configure: *Select format:* Input Sample Period; *Sample Period (Clock Cycles):* 1024. This would result in the same configuration!
</div> 
 

* In the **Block Diagram** disconnect the filter's **aclk** from the 44,1KHz clock and connect it to the **45MHz** clock. The idea is to have the filter running much faster than the input sample rate in order to allow the tools to choose a serial implementatin with a single DSP.


* **Validate the Design** and then **Save** it. Select **Run Implementation** and follow all steps to start this process.


* When it finishes click **Cancel** and check **Project Summary** again in the *Project Manager* view. Note that only **1 DSP** is now used to implement the same 32 coefficient filter because the tool is not taking advantage of the available time between new samples.


<div class="alert alert-block alert-info">
<b>Info:</b> Note that the sample rate is 1024 times slower than the clock rate and thus, this filter could have 1024 coefficients and still be implemented with a single DSP. In fact, if coefficients are symmetric, the filter could have 2048 coefficients and still be implemented with a single DSP.
</div>


* **Open Block Design** again and double-click the **clock_synth** module. Add another **Slice** and **Utility Buffer** to create a clock with a frequency of 88,2KHz (take the 9th bit of the counter). Name this output as **clk88K2**.


* **Validate the Design** and then **Regenerate Layout** and **save** it.


* In the **Block Diagram** disconnect the filter's **aclk** from the **clk45M** clock and connect it to the **clk88K2** clock.


* Double-click the **FIR Compiler**.  In the **Channel Specification** tab configure the **Hardware Oversampling Specification**. Change the Clock Frequency to **0.0882 MHz** and click **OK**. 


* **Validate the Design** and then **Save** it. Select **Run Implementation** and check **Project Summary** when implementation is complete. You should see that the filter is now implemented with **9 DSPs**. Can you explain why?





### Step 3.3 ###

Explore FIR channels and coefficient loading.

* Using Matlab **fdatool** create a **bandpass filter** centered at 5kHz, as shown in the figure below. Quantize the coefficients as **Fixed-point**, **Full** precision, coefficients with **16 bits** and **14 bits** fractional length, scaled to fully utilize the dynamic range, and input word length limited to **16 bits** with **14 bits** fractional length. Export as **NumBP*.

![Figure](../lab4/lab4_figures/fig9.png)


* Design also a **lowpass filter** similar to the one provided in this lab (coefs.coe file), using the configurations shown in the figure below. Quantize the coefficients as you have done for the band-pass filter and export as **NumLP*.


![Figure](../lab4/lab4_figures/fig10.png)


* Save the coefficients in two different data files using Matlab command line:
```
>> csvwrite('NumLP.dat',floor(NumLP*2^14));
>> csvwrite('NumBP.dat',floor(NumBP*2^14));
```


* Create a new file using Notepad++ or other text editor to generate a new **COE** file with the coefficients of both filters in sequence, first the band-pass and then the low-pass. Name it **coefsBPLP.coe** and save it in the **{sources}\lab4** directory. The file should look like this:

```
radix=10;
coefdata=10908,6825,8756,10873,13136,15510,17938,20366,22731,24970,27032,28845,30361,31551,32354,32768,32768,32354,31551,30361,28845,27032,24970,22731,20366,17938,15510,13136,10873,8756,6825,10908,-896,618,-970,-2076,-1318,2468,8286,12234,9530,-1496,-16562,-26718,-23836,-6802,16366,32768,32768,16366,-6802,-23836,-26718,-16562,-1496,9530,12234,8286,2468,-1318,-2076,-970,618,-896;
```


<div class="alert alert-block alert-info">
<b>Info:</b> All coefficients sets in a multiple set implementation must exhibit the same symmetry. For example, if even one set of a multi-set has non-symmetric coefficient structure, then all sets are implemented using that structure. All coefficient sets must also be of the same vector length. If one coefficient set has fewer coefficients, it must be zero padded.
</div>


* **Open Block Design** and double-click the **FIR Compiler**. In the **Filter Options** tab change the *Coefficient File* to the one with both filters (**coefsBPLP.coe**). Wait until the file is updated and then change:
    * Filter Options tab > Number of Coefficients Sets = 2
    * Channel Specification tab > Hardware Oversampling Specification > Sample Period (Clock Cycles):1
    * Implementation tab > Coefficient Options > Coefficient Type : Signed
    * Implementation tab > Coefficient Options > Quantization : Maximize Dynamic Range
    * Implementation tab > Coefficient Options > Coefficient Width : 16
    * Implementation tab > Coefficient Options > Coefficient Fractional Bits = 0
    
    
* In the left-hand window of the **Re-customize IP** window you can see the IP Symbol and also the **Freq. Response** for each of the filters implemented with this coefficient's set. As expected, **set 1** is a band-pass filter while **set 2** is a low-pass filter.


* Click **OK** an go back to the **Block Design**. Make sure the FIR compiler clock pin is connected to the 44100Hz clock.


* Add a **constant** and a **concat** IP cores. Configure the Constant to be **7 bits width** and **value 0**. The Concat module is supposed to concatenate an external bit comming from a switch with the 7 bits set to zero. Configure it accordingly. This will create a **config vector** equal to **0000 0000** or **0000 0001**, which will enable us to select the filter's coefficients.

* **Validate the Design** and then **Save** it. You should have the following: 

![Figure](../lab4/lab4_figures/fig11.png)


* Edit the constraints file in order to define the switch location. To do that just uncomment **line 5**. Save the file and select **Generate Bitstream**. 


* **Programm** the board and check the filter's output for diferent coefficients's set (using ILA). You can choose the low-pass filter or the band-pass filter, by switching **switch 0**.


* Close **Hardware Manager** and **turn-off** the board.



